# Reading, Writing and Validating Data in PySpark HW

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HW1').getOrCreate()

## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [4]:
path = "Datasets/"
df = spark.read.csv(path+'pga_tour_historical.csv',header=True,inferSchema=True)

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [5]:
df.show(5)

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows



In [7]:
df.head(5)

[Row(Player Name='Robert Garrigus', Season=2010, Statistic='Driving Distance', Variable='Driving Distance - (ROUNDS)', Value='71'),
 Row(Player Name='Bubba Watson', Season=2010, Statistic='Driving Distance', Variable='Driving Distance - (ROUNDS)', Value='77'),
 Row(Player Name='Dustin Johnson', Season=2010, Statistic='Driving Distance', Variable='Driving Distance - (ROUNDS)', Value='83'),
 Row(Player Name='Brett Wetterich', Season=2010, Statistic='Driving Distance', Variable='Driving Distance - (ROUNDS)', Value='54'),
 Row(Player Name='J.B. Holmes', Season=2010, Statistic='Driving Distance', Variable='Driving Distance - (ROUNDS)', Value='100')]

## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [8]:
df.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [9]:
from pyspark.sql.types import *

In [10]:
data_schema = [
    StructField("Player_Name", StringType(),True),
    StructField("Season", IntegerType(),True),
    StructField("Statistic", StringType(),True),
    StructField("Variable", StringType(),True),
    StructField("Value", IntegerType(),True),
    ]

final_struct = StructType(fields=data_schema)

In [11]:
df = spark.read.csv(path+'pga_tour_historical.csv',schema=final_struct)

In [12]:
df.printSchema()

root
 |-- Player_Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: integer (nullable = true)



## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [16]:
df.select("Value").describe().show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|           1657247|
|   mean|12494.388998743096|
| stddev|157274.75673570848|
|    min|              -178|
|    max|           3564954|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [14]:
df.select("Value","Season").summary('min','max').show()

+-------+-------+------+
|summary|  Value|Season|
+-------+-------+------+
|    min|   -178|  2010|
|    max|3564954|  2018|
+-------+-------+------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [22]:
extract = df.select("Value","Season")
extract.write.mode('overwrite').parquet("partition_parquet/")

## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [23]:
extract.write.mode('overwrite').partitionBy('Season').parquet("partition_parquet/")

## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [24]:
parquet = spark.read.parquet('partition_parquet/')

In [25]:
parquet.show(5)

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
+-----+------+
only showing top 5 rows



## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [26]:
path_parquet = "partition_parquet/"

part_df = spark.read.option("basePath",path_parquet).parquet(
    path_parquet+'Season=2010/',
    path_parquet+'Season=2011/',
    path_parquet+'Season=2012/',
)

In [27]:
part_df.groupBy("Season").count().show()

+------+------+
|Season| count|
+------+------+
|  2012|311658|
|  2010|323083|
|  2011|305112|
+------+------+



## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [18]:
pets = spark.createDataFrame([
    ['Ginko','cat',2,'06-06-2019'],
    ['Alistair','dog',3,'02-19-2018'],
    ['Noiraude','cat',10,'11-23-2010'],
    ],['name','type','age','birth'])

In [20]:
pets.show()

+--------+----+---+----------+
|    name|type|age|     birth|
+--------+----+---+----------+
|   Ginko| cat|  2|06-06-2019|
|Alistair| dog|  3|02-19-2018|
|Noiraude| cat| 10|11-23-2010|
+--------+----+---+----------+



In [21]:
pets.printSchema()

root
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- age: long (nullable = true)
 |-- birth: string (nullable = true)



## We're done! Great job!